In [ ]:
import math
import numpy as np
import pandas as pd
import lightningchart as lc
from scipy.stats import gaussian_kde
from skimage.measure import find_contours
from scipy.interpolate import griddata

lc.set_license('my-license-key')

file_path = "../Dataset/gold_recovery_full_with_profit.csv"
df = pd.read_csv(file_path)

df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y', errors='coerce')

# Create a 'time' column using the DataFrame index for time-series charts.
df['time'] = df.index.astype(float)

# Fill or convert columns used by multiple charts.
numeric_cols = [
    'final.output.concentrate_ag', 'final.output.concentrate_pb',
    'final.output.concentrate_sol','final.output.concentrate_au',
    'final.output.recovery', 'profit'
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# Create a 'TrialNumber' column for the profit distribution chart.
df["TrialNumber"] = df.index.astype(float)


# 2. CREATE A SINGLE DASHBOARD WITH 5 ROWS × 8 COLUMNS

dashboard = lc.Dashboard(rows=5, columns=8, theme=lc.Themes.Light)


# CHART A: Final Features: Ag, Pb, Sol, Au
#   row=0, column=0, row_span=3, column_span=2

chart_final = dashboard.ChartXY(
    title='Final Features: Ag, Pb, Sol, Au',
    row_index=0, column_index=0,
    row_span=3, column_span=2
)
chart_final.get_default_y_axis().dispose()  # Remove the default y-axis
legend_final = chart_final.add_legend().set_dragging_mode("draggable")

final_features = [
    ('final.output.concentrate_ag', 'Ag'),
    ('final.output.concentrate_pb', 'Pb'),
    ('final.output.concentrate_sol', 'Sol'),
    ('final.output.concentrate_au', 'Au')
]

for i, (col, label) in enumerate(final_features):
    axis_y = chart_final.add_y_axis(stack_index=i)
    axis_y.set_title(title=label)

    series = chart_final.add_line_series(y_axis=axis_y, data_pattern='ProgressiveX')
    x_vals = df['time'].tolist()
    y_vals = df[col].tolist()
    series.append_samples(x_values=x_vals, y_values=y_vals)
    series.set_name(label)
    legend_final.add(series)

chart_final.get_default_x_axis().set_interval(0, df['time'].max())
chart_final.get_default_x_axis().set_title('Time (Index)')

zbc = dashboard.ZoomBandChart(chart=chart_final, row_index=3, column_index=0,
    row_span=1, column_span=2)
zbc.add_series(series)


# CHART B: Correlation with Profit Delta
#   row=3, column=0, row_span=2, column_span=2

# Compute deltas for correlation analysis
df['profit_delta'] = df['profit'].diff()
features_dict = {
    "final.output.concentrate_ag": "ag",
    "final.output.concentrate_pb": "pb",
    "final.output.concentrate_sol": "sol",
    "final.output.concentrate_au": "au"
}
for full_name, short_label in features_dict.items():
    df[full_name + "_delta"] = df[full_name].diff()

df_deltas = df.dropna(axis=0)
correlation_data = []
for full_name, short_label in features_dict.items():
    corr = df_deltas[full_name + "_delta"].corr(df_deltas["profit_delta"])
    correlation_data.append({"category": short_label, "value": float(corr)})

correlation_data_sorted = sorted(correlation_data, key=lambda d: abs(d["value"]), reverse=True)

chart_corr = dashboard.BarChart(
    vertical=True,
    row_index=4, column_index=0,
    row_span=1, column_span=2,
).set_title("Correlation with Profit Delta").set_title_font(size=12)
chart_corr.set_sorting('disabled')
chart_corr.set_data(correlation_data_sorted)
chart_corr.set_label_rotation(45)
chart_corr.set_palette_colors(
    steps=[
        {'value': 0,   'color': lc.Color('blue')},
        {'value': 0.5, 'color': lc.Color('yellow')},
        {'value': 1,   'color': lc.Color('red')}
    ]
)

# CHART C: Profit Distribution & Range Boundaries
#   row=0, column=2, row_span=2, column_span=3

df_copy = df.iloc[:2000]

chart_profit_dist = dashboard.ChartXY(
    row_index=0, column_index=2,
    row_span=2, column_span=3
).set_title(title="Profit Distribution and Range Boundaries Across First 2000 Trial Intervals")

point_series = chart_profit_dist.add_point_series()
point_series.set_point_color(lc.Color(0, 128, 255)).set_draw_order(2)
point_series.set_point_size(3)
point_series.add(
    x=df_copy["TrialNumber"].tolist(),
    y=df_copy["profit"].tolist()
)

bound_series = chart_profit_dist.add_rectangle_series().set_draw_order(0)
box_series = chart_profit_dist.add_rectangle_series().set_draw_order(1)

interval_size = 100
max_trial = df_copy["TrialNumber"].max()
num_intervals = int(math.ceil(max_trial / interval_size))

for i in range(num_intervals):
    x_start = i * interval_size
    x_end = (i + 1) * interval_size
    
    # Subset data for this interval.
    subdf = df_copy[(df_copy["TrialNumber"] >= x_start) & (df_copy["TrialNumber"] < x_end)]
    if subdf.empty:
        continue

    min_profit = subdf["profit"].min()
    max_profit = subdf["profit"].max()

    # Large pink bounding box covering the entire interval range vertically.
    outer_rect = bound_series.add(x_start, min_profit, x_end, max_profit)
    outer_rect.set_color(lc.Color(255, 192, 203, 128))  # pink, semi-transparent

    # For each point in the interval, add a red rectangle spanning the entire width (x_start to x_end).
    # Only the vertical band is small around each profit point.
    half_height = (max_profit - min_profit) * 0.02
    for _, row in subdf.iterrows():
        py = row["profit"]
        y1 = py - half_height
        y2 = py + half_height
        
        small_rect = box_series.add(x_start, y1, x_end, y2)
        # Make these rectangles semi-transparent red so they layer nicely.
        small_rect.set_color(lc.Color(255, 153, 153, 128))  # RGBA

chart_profit_dist.get_default_x_axis().set_title("Trial Number")
chart_profit_dist.get_default_y_axis().set_title("Profit")

# CHART D: Bubble Chart: Recovery vs. Concentrate Au with Profit
#   row=0, column=5, row_span=2, column_span=3

chart_bubble = dashboard.ChartXY(
    row_index=2, column_index=2,
    row_span=3, column_span=4,
    title="Recovery vs. Concentrate Au with Profit"
)

# Prepare data for bubble chart
x_bubble = df["final.output.concentrate_au"].tolist()
y_bubble = df["final.output.recovery"].tolist()
profit_bubble = df["profit"].tolist()

min_size, max_size = min(profit_bubble), max(profit_bubble)
# Scale bubble sizes from 10 to 60
scaled_sizes = [(20 * (p - min_size) / (max_size - min_size)) + 5 if max_size > min_size else 5
                for p in profit_bubble]
# Normalize profit for color lookup [0..1]
normalized_profits = [(p - min_size) / (max_size - min_size) if max_size > min_size else 0
                      for p in profit_bubble]

series_bubble = chart_bubble.add_point_series(sizes=True, lookup_values=True)
series_bubble.append_samples(
    x_values=x_bubble,
    y_values=y_bubble,
    sizes=scaled_sizes,
    lookup_values=normalized_profits
)
series_bubble.set_individual_point_color_enabled(True)
series_bubble.set_palette_point_coloring(
    steps=[
        {"value": 0.0,  "color": lc.Color('green')},
        {"value": 0.25, "color": lc.Color('yellow')},
        {"value": 0.75, "color": lc.Color('orange')},
        {"value": 1.0,  "color": lc.Color('red')}
    ],
    look_up_property="value",
    percentage_values=True
)
chart_bubble.get_default_x_axis().set_title("Final Output Concentrate Au")
chart_bubble.get_default_y_axis().set_title("Final Output Recovery")
chart_bubble.add_legend(data=series_bubble).set_title("Profit").set_dragging_mode("draggable").set_font_size(10)


# CHART E: KDE: Final Concentrate Gold vs. Recovery
#   row=2, column=2, row_span=3, column_span=3

chart_kde = dashboard.ChartXY(
    row_index=0, column_index=5,
    row_span=2, column_span=3,
    title="KDE: Final Concentrate Gold vs. Recovery"
)

x_kde = df["final.output.concentrate_au"].values
y_kde = df["final.output.recovery"].values

x_min, x_max = x_kde.min() - 0.5, x_kde.max() + 0.5
y_min, y_max = y_kde.min() - 5, y_kde.max() + 5

xy = np.vstack([x_kde, y_kde])
kde_func = gaussian_kde(xy)

num_bins_x = 100
num_bins_y = 100
X_values = np.linspace(x_min, x_max, num_bins_x)
Y_values = np.linspace(y_min, y_max, num_bins_y)

kde_array = np.zeros((num_bins_y, num_bins_x), dtype=float)
for i, y_val in enumerate(Y_values):
    for j, x_val in enumerate(X_values):
        kde_array[i, j] = kde_func([x_val, y_val])

heatmap_series = chart_kde.add_heatmap_grid_series(
    columns=num_bins_x, rows=num_bins_y, data_order="rows"
)
heatmap_series.set_start(x=x_min, y=y_min)
heatmap_series.set_end(x=x_max, y=y_max)
heatmap_series.set_step(
    x=(x_max - x_min)/num_bins_x,
    y=(y_max - y_min)/num_bins_y
)
heatmap_series.set_name("Density")
heatmap_series.invalidate_intensity_values(kde_array.tolist())

kde_min = kde_array.min()
kde_max = kde_array.max()
palette_steps = [
    {"value": float(kde_min), "color": lc.Color(230, 247, 255)},
    {"value": float(kde_max), "color": lc.Color(0, 0, 255)}
]
heatmap_series.set_palette_coloring(
    steps=palette_steps,
    look_up_property="value",
    interpolate=True
)
heatmap_series.hide_wireframe()

# Add contour lines
levels = 10
density_levels = np.linspace(kde_min, kde_max, levels)
for lvl in density_levels:
    contours = find_contours(kde_array, level=lvl)
    for contour in contours:
        real_x = []
        real_y = []
        for (row_idx, col_idx) in contour:
            x_coord = x_min + (col_idx / num_bins_x) * (x_max - x_min)
            y_coord = y_min + (row_idx / num_bins_y) * (y_max - y_min)
            real_x.append(x_coord)
            real_y.append(y_coord)
        line_series = chart_kde.add_line_series(data_pattern="ProgressiveX")
        line_series.set_line_color(lc.Color("white"))
        line_series.set_line_thickness(2)
        line_series.add(real_x, real_y)

point_series_kde = chart_kde.add_point_series().set_point_size(2)
point_series_kde.set_point_color(lc.Color(128, 128, 128)).set_draw_order(2)
point_series_kde.add(x_kde.tolist(), y_kde.tolist())

chart_kde.get_default_x_axis().set_title("Final Concentrate Gold (Au)")
chart_kde.get_default_y_axis().set_title("Recovery (%)")
chart_kde.add_legend(data=heatmap_series, horizontal=True).set_title("Density").set_margin(right=180).set_dragging_mode("draggable").set_font_size(10)


# CHART F: 3D Chart => Profit vs. Concentrate Au & Recovery
#   row=2, column=6, row_span=3, column_span=2

chart3d = dashboard.Chart3D(
    row_index=2, column_index=6,
    row_span=3, column_span=2,
    title="Profit vs. Concentrate Au & Recovery"
)

# Prepare data for 3D surface
df_3d = df.dropna(subset=["final.output.concentrate_au", "final.output.recovery", "profit"]).reset_index(drop=True)
x_3d = df_3d["final.output.concentrate_au"].to_numpy()
z_3d = df_3d["final.output.recovery"].to_numpy()
y_3d = df_3d["profit"].to_numpy()

num_bins_x_3d = 50
num_bins_z_3d = 50
xi = np.linspace(x_3d.min(), x_3d.max(), num_bins_x_3d)
zi = np.linspace(z_3d.min(), z_3d.max(), num_bins_z_3d)
XI, ZI = np.meshgrid(xi, zi)

# Interpolate scattered profit data onto the grid
height_map = griddata((x_3d, z_3d), y_3d, (XI, ZI), method='cubic')
height_map = np.nan_to_num(height_map, nan=0)

surface_series = chart3d.add_surface_grid_series(
    columns=num_bins_x_3d, rows=num_bins_z_3d, data_order="rows"
)
surface_series.set_start(x=xi.min(), z=zi.min())
surface_series.set_end(x=xi.max(), z=zi.max())
surface_series.set_step(
    x=(xi.max()-xi.min()) / num_bins_x_3d,
    z=(zi.max()-zi.min()) / num_bins_z_3d
)
surface_series.invalidate_height_map(height_map.tolist())
surface_series.hide_wireframe()

# Color palette for profit
data_min_3d = float(height_map.min())
data_max_3d = float(height_map.max())
data_mid_3d = (data_min_3d + data_max_3d) / 2

palette_steps_3d = [
    {"value": data_min_3d, "color": lc.Color("blue")},
    {"value": data_mid_3d, "color": lc.Color("white")},
    {"value": data_max_3d, "color": lc.Color("red")},
]
surface_series.set_palette_coloring(
    steps=palette_steps_3d,
    look_up_property="value",
    percentage_values=False
)
surface_series.invalidate_intensity_values(height_map.tolist())

chart3d.get_default_x_axis().set_title("Final Output Concentrate Au").set_title_font(size=12)
chart3d.get_default_y_axis().set_title("Profit").set_title_font(size=12)
chart3d.get_default_z_axis().set_title("Final Output Recovery").set_title_font(size=12)
chart3d.set_camera_location(x=5, y=5, z=5)
chart3d.add_legend(data=surface_series, horizontal=True).set_title("Profit").set_margin(bottom=400).set_dragging_mode("draggable").set_font_size(10)


# 3. OPEN THE DASHBOARD

dashboard.open(method="browser")


C:\Users\Soroush\AppData\Local\Temp\ipykernel_1988\3244162560.py:242: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  kde_array[i, j] = kde_func([x_val, y_val])
127.0.0.1 - - [17/Mar/2025 17:48:15] "GET / HTTP/1.1" 200 -
